In [1]:
import pandas as pd 
import numpy as np 
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import re

#### Read the Dataset


In [2]:
df = pd.read_csv('toxic-comments.csv')

In [3]:
df.shape

(159571, 8)

In [4]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [5]:
#Define target variables(adapt based on your need)
toxicities = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

### Text Cleaning

In [6]:
def clean_text(text):
    text = text.lower() #lowercase
    text = re.sub(r"[^a-zA-Z0-9]+","", text) # Remove non alphanumeric 
    return text

In [7]:
df["comment_text"] = df["comment_text"].apply(clean_text)

In [8]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanationwhytheeditsmadeundermyusernamehardc...,0,0,0,0,0,0
1,000103f0d9cfb60f,dawwhematchesthisbackgroundcolourimseeminglyst...,0,0,0,0,0,0
2,000113f07ec002fd,heymanimreallynottryingtoeditwaritsjustthatthi...,0,0,0,0,0,0
3,0001b41b1c6bb37e,moreicantmakeanyrealsuggestionsonimprovementiw...,0,0,0,0,0,0
4,0001d958c54c6e35,yousiraremyheroanychanceyourememberwhatpagetha...,0,0,0,0,0,0


### Define input and output variable

In [9]:
#Feature ad target preparation
comments = df["comment_text"].tolist()
targets = df[toxicities].values

In [10]:
targets.shape

(159571, 6)

### Prepare the Data

In [11]:
# Tokenization and Padding
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(comments)
sequences = tokenizer.texts_to_sequences(comments)
padded_sequences = pad_sequences(sequences, maxlen=200)

In [12]:
padded_sequences.shape

(159571, 200)

### Cross Validate

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences, targets, test_size=0.2, random_state=0
)

In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((127656, 200), (127656, 6), (31915, 200), (31915, 6))

### Build the Model

In [15]:
model = Sequential()
model.add(Embedding(5000, 128, input_length=200))
model.add(GRU(64))
model.add(Dense(6, activation="sigmoid"))

In [16]:
from keras.utils import plot_model

In [17]:
#plot_model(model, show_dtype=True, show_layer_activations=True, show_layer_names=True, show_shapes=True)

### Compile the Model

In [18]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=['accuracy'])

### Train the Model

In [19]:
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/3


3990/3990 [==============================] - 467s 116ms/step - loss: 0.1433 - accuracy: 0.9871 - val_loss: 0.1425 - val_accuracy: 0.9936
Epoch 2/3
3990/3990 [==============================] - 474s 119ms/step - loss: 0.1384 - accuracy: 0.9912 - val_loss: 0.1420 - val_accuracy: 0.9888
Epoch 3/3
3990/3990 [==============================] - 439s 110ms/step - loss: 0.1371 - accuracy: 0.9920 - val_loss: 0.1462 - val_accuracy: 0.9936


### Predict on new data

In [45]:
#new_comment = "This is an aweful and offensive comment!"
#new_comment = "he studied good so he won"
new_comment = "Good to see you back on social media."

In [46]:

sequences1 = tokenizer.texts_to_sequences(clean_text(new_comment))
padded_sequences1 = pad_sequences(sequences1, maxlen=200)

In [47]:
prediction = model.predict(padded_sequences1)[0]

1/1 [==============================] - 0s 60ms/step


In [48]:
prediction


array([0.09300507, 0.01090561, 0.05616373, 0.00272231, 0.03965   ,
       0.00543076], dtype=float32)

In [39]:
#new_comment = "he studied good so he won"
for toxicity, prob in zip(toxicities, prediction):
    print(f"Toxicity: {toxicity}, Prediction: {prob:.2f}")

Toxicity: toxic, Prediction: 0.09
Toxicity: severe_toxic, Prediction: 0.01
Toxicity: obscene, Prediction: 0.06
Toxicity: threat, Prediction: 0.00
Toxicity: insult, Prediction: 0.04
Toxicity: identity_hate, Prediction: 0.01


In [44]:
#new_comment = "This is an aweful and offensive comment!"
for toxicity, prob in zip(toxicities, prediction):
    print(f"Toxicity: {toxicity}, Prediction: {prob:.2f}")

Toxicity: toxic, Prediction: 0.99
Toxicity: severe_toxic, Prediction: 0.27
Toxicity: obscene, Prediction: 0.88
Toxicity: threat, Prediction: 0.09
Toxicity: insult, Prediction: 0.87
Toxicity: identity_hate, Prediction: 0.20


In [49]:
#new_comment = "Good to see you back on social media."
for toxicity, prob in zip(toxicities, prediction):
    print(f"Toxicity: {toxicity}, Prediction: {prob:.2f}")

Toxicity: toxic, Prediction: 0.09
Toxicity: severe_toxic, Prediction: 0.01
Toxicity: obscene, Prediction: 0.06
Toxicity: threat, Prediction: 0.00
Toxicity: insult, Prediction: 0.04
Toxicity: identity_hate, Prediction: 0.01
